### Hugging face eval cnn-dm
+ 使用 cnn dailymail 通过 Hugging Face 的各个模型效果评估
+ https://github.com/hellotransformers/Natural_Language_Processing_with_Transformers
+ https://xiaosheng.run/2022/03/29/transformers-note-8.html

In [47]:
# !pip install datasets==2.5.2
# !pip uninstall transformers
# !pip install transformers # 需要 3.1.0, 4.x 会报错
# !export http_proxy='http://172.19.57.45:3128/'
# !export http_proxy='http://172.19.57.45:3128/'
# !export http_proxy=''
# !export http_proxy=''

# 数据准备
+ 云端直接下载有问题
+ 远程加载 可以参考 https://github.com/huggingface/datasets/issues/996
+ 本地加载 可以参考 https://blog.csdn.net/PolarisRisingWar/article/details/124042709

In [48]:
# 代理必须关闭
# 服务器上也需要关闭代理
# hide_output
import datasets
from datasets import load_dataset

# 远程加载
# dataset = load_dataset("cnn_dailymail",  version="3.0.0") # 有bug
# dataset = load_dataset("ccdv/cnn_dailymail",  version="3.0.0")
# 本地加载
dataset = datasets.load_from_disk('local_cnn-dm')
print(f"Features: {dataset['train'].column_names}")

Features: ['article', 'highlights', 'id']


In [49]:
sample = dataset["train"][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])



Article (excerpt of 500 characters, total length: 3192):

(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has n

Summary (length: 180):
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .


In [50]:
sample_text = dataset["train"][1]["article"][:2000]
print(sample_text)
# We'll collect the generated summaries of each model in a dictionary
summaries = {}

(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics. Victory was never seriou

In [51]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /home/users/sunhongchao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# baseline

In [52]:
def three_sentence_summary(text): 
	return "\n".join(sent_tokenize(text)[:3]) 
summaries["baseline"] = three_sentence_summary(sample_text)

# gpt2

In [53]:
from transformers import pipeline, set_seed 
set_seed(42) 
pipe = pipeline("text-generation", model="gpt2-xl") 
gpt2_query = sample_text + "\nTL;DR:\n" 
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)

summaries["gpt2"] = "\n".join( sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


# t5

In [54]:
pipe = pipeline("summarization", model="t5-large") 
pipe_out = pipe(sample_text) 
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

# bart

In [55]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn") 
pipe_out = pipe(sample_text) 
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

# pegasus

In [56]:
# pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail") 
# pipe_out = pipe(sample_text) 
# summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

# 不同模型效果对比

In [57]:
print("GROUND TRUTH") 
print(dataset["train"][1]["highlights"]) 
print("") 
for model_name in summaries: 
	print(model_name.upper()) 
	print(summaries[model_name]) 
	print("")

GROUND TRUTH
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .

BASELINE
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay.
The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds.
The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover.

GPT2
The Olympic champion wins gold in Moscow.
Bolt gets back in the picture with an impressive silver.
A triple by Fraser-Pryce, Bailey-Cole, and Carter leaves the Jamaicans in the lead for the last part of the relay.
Bolt dominates the final 50 meters and finishes off his hat trick and completes his double by completi

# 评估指标

In [58]:
from datasets import load_metric 
bleu_metric = load_metric("sacrebleu")

In [59]:
import pandas as pd 
import numpy as np 
bleu_metric.add( prediction="the the the the the the", reference=["the cat is on the mat"]) 
results = bleu_metric.compute(smooth_method="floor", smooth_value=0) 
results["precisions"] = [np.round(p, 2) for p in results["precisions"]] 
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,1
sys_len,6
ref_len,6


In [60]:
bleu_metric.add( prediction="the cat is on mat", reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0) 
results["precisions"] = [np.round(p, 2)for p in results["precisions"]] 
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,57.893
counts,"[5, 3, 2, 1]"
totals,"[5, 4, 3, 2]"
precisions,"[100.0, 75.0, 66.67, 50.0]"
bp,0.818731
sys_len,5
ref_len,6


In [61]:
# rouge_score==0.0.4 work well
rouge_metric = load_metric("rouge")

In [62]:
reference = dataset["train"][1]["highlights"] 
records = [] 
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"] 
for model_name in summaries: 
	rouge_metric.add(prediction=summaries[model_name], reference=reference) 
	score = rouge_metric.compute() 
	rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
	records.append(rouge_dict) 
pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.303571,0.090909,0.214286,0.232143
gpt2,0.244898,0.000000,0.122449,0.224490
t5,0.472222,0.228571,0.388889,0.472222
bart,0.582278,0.207792,0.455696,0.506329
